# Rockset

⚠️ **弃用通知：Rockset 集成已禁用**
> 
> 自 2024 年 6 月起，Rockset 已被 [OpenAI 收购](https://openai.com/index/openai-acquires-rockset/) 并**关闭了其公共服务**。
> 
> Rockset 是一款实时分析数据库，以其世界一流的索引和检索能力而闻名。如今，其核心团队和技术正在整合到 OpenAI 的基础设施中，为未来的 AI 产品提供支持。
> 
> 此 LangChain 集成已不再可用，仅**用于存档目的**。

> Rockset 是一个实时分析数据库，能够对海量半结构化数据进行查询，而无需承担运营负担。使用 Rockset，摄入的数据可在1秒内查询，而针对该数据的分析查询通常在毫秒内执行。Rockset 是经过计算优化的，因此适合服务于 100TB 以下范围（或通过 rollups 可服务于大于 100s TB 的数据）的高并发应用。

本笔记本演示了如何将 Rockset 用作 langchain 中的文档加载器。要开始使用，请确保您拥有 Rockset 账户和 API 密钥。

## 设置环境

1. 前往[Rockset 控制台](https://console.rockset.com/apikeys)并获取 API 密钥。请从[API 参考](https://rockset.com/docs/rest-api/#introduction)中找到您的 API 区域。在本笔记本中，我们将假设您正在使用来自 `Oregon(us-west-2)` 的 Rockset。
2. 设置环境变量 `ROCKSET_API_KEY`。
3. 安装 Rockset python 客户端，Langchain 将使用它与 Rockset 数据库进行交互。

In [ ]:
%pip install --upgrade --quiet  rockset

# 加载文档
Rockset 与 LangChain 的集成允许您使用 SQL 查询从 Rockset 集合加载文档。要做到这一点，您必须构建一个 `RocksetLoader` 对象。下面是一个初始化 `RocksetLoader` 的示例片段。

In [ ]:
from langchain_community.document_loaders import RocksetLoader
from rockset import Regions, RocksetClient, models

loader = RocksetLoader(
    RocksetClient(Regions.usw2a1, "<api key>"),
    models.QueryRequestSql(query="SELECT * FROM langchain_demo LIMIT 3"),  # SQL query
    ["text"],  # content columns
    metadata_keys=["id", "date"],  # metadata columns
)

在此，您可以查看正在运行的以下查询：

```sql
SELECT * FROM langchain_demo LIMIT 3
```

集合中的 `text` 列用作页面内容，以及记录的 `id` 和 `date` 列用作元数据（如果您未将任何内容传递给 `metadata_keys`，则整个 Rockset 文档将用作元数据）。

要执行查询并访问生成的 `Document` 的迭代器，请运行：

In [ ]:
loader.lazy_load()

要执行查询并一次性访问所有结果 `Document`，请运行：

In [ ]:
loader.load()

`loader.load()` 的示例如下响应：
```python
[
    Document(
        page_content="Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas a libero porta, dictum ipsum eget, hendrerit neque. Morbi blandit, ex ut suscipit viverra, enim velit tincidunt tellus, a tempor velit nunc et ex. Proin hendrerit odio nec convallis lobortis. Aenean in purus dolor. Vestibulum orci orci, laoreet eget magna in, commodo euismod justo.",
        metadata={"id": 83209, "date": "2022-11-13T18:26:45.000000Z"}
    ),
    Document(
        page_content="Integer at finibus odio. Nam sit amet enim cursus lacus gravida feugiat vestibulum sed libero. Aenean eleifend est quis elementum tincidunt. Curabitur sit amet ornare erat. Nulla id dolor ut magna volutpat sodales fringilla vel ipsum. Donec ultricies, lacus sed fermentum dignissim, lorem elit aliquam ligula, sed suscipit sapien purus nec ligula.",
        metadata={"id": 89313, "date": "2022-11-13T18:28:53.000000Z"}
    ),
    Document(
        page_content="Morbi tortor enim, commodo id efficitur vitae, fringilla nec mi. Nullam molestie faucibus aliquet. Praesent a est facilisis, condimentum justo sit amet, viverra erat. Fusce volutpat nisi vel purus blandit, et facilisis felis accumsan. Phasellus luctus ligula ultrices tellus tempor hendrerit. Donec at ultricies leo.",
        metadata={"id": 87732, "date": "2022-11-13T18:49:04.000000Z"}
    )
]
```

## 使用多列作为内容

您可以选择使用多列作为内容：

In [ ]:
from langchain_community.document_loaders import RocksetLoader
from rockset import Regions, RocksetClient, models

loader = RocksetLoader(
    RocksetClient(Regions.usw2a1, "<api key>"),
    models.QueryRequestSql(query="SELECT * FROM langchain_demo LIMIT 1 WHERE id=38"),
    ["sentence1", "sentence2"],  # TWO content columns
)

假设 "sentence1" 字段为 `"This is the first sentence."`，"sentence2" 字段为 `"This is the second sentence."`，则生成的 `Document` 的 `page_content` 将会是：

```
This is the first sentence.
This is the second sentence.
```

您可以通过在 `RocksetLoader` 的构造函数中设置 `content_columns_joiner` 参数来定义自己的函数来连接内容列。`content_columns_joiner` 是一个接收 `List[Tuple[str, Any]]]` 作为参数的方法，代表一个 (列名, 列值) 的元组列表。默认情况下，这是一个将每个列值用新行连接的方法。

例如，如果您想用空格而不是新行来连接 sentence1 和 sentence2，您可以这样设置 `content_columns_joiner`：

In [ ]:
RocksetLoader(
    RocksetClient(Regions.usw2a1, "<api key>"),
    models.QueryRequestSql(query="SELECT * FROM langchain_demo LIMIT 1 WHERE id=38"),
    ["sentence1", "sentence2"],
    content_columns_joiner=lambda docs: " ".join(
        [doc[1] for doc in docs]
    ),  # join with space instead of /n
)

`Document` 的 `page_content` 将是：

```
This is the first sentence. This is the second sentence.
```

很多时候你希望在 `page_content` 中包含列名。你可以这样做：

In [ ]:
RocksetLoader(
    RocksetClient(Regions.usw2a1, "<api key>"),
    models.QueryRequestSql(query="SELECT * FROM langchain_demo LIMIT 1 WHERE id=38"),
    ["sentence1", "sentence2"],
    content_columns_joiner=lambda docs: "\n".join(
        [f"{doc[0]}: {doc[1]}" for doc in docs]
    ),
)

这将导致以下 `page_content`：

```
sentence1: 这是第一句话。
sentence2: 这是第二句话。
```